# 행정안전부 도로명주소 API 
### 건물명으로부터 도로명 주소 구하기

In [1]:
import requests
from urllib.parse import urlparse, quote
import pandas as pd

In [3]:
with open('roadapikey.txt',mode='r') as key_file:
    road_key = key_file.read()

* URL을 만드는 규칙  
path가 먼저 나오고 그다음에? 그리고 parameter(항목 = 값 & 다음항목 = 값 & ...)

In [28]:
len(road_key)

43

In [ ]:
# base64인코딩을 해줘야 할 때도 있음.
quote("한국경제신문사")

In [9]:
# API활용가이드 참고
road_url = "https://www.juso.go.kr/addrlink/addrLinkApi.do"         

url = f'{road_url}?confmKey={road_key}&currentPage=1&countPerPage=10&keyword={quote("한국경제신문사")}&resultType=json'

In [8]:
result = requests.get(url).json()      # REST API키를 줘서 받겠다. 
result                                 
# errorMessage : 정상 이면 정상! -> API활용가이드 보면 메세지가 나와있다. 

{'results': {'common': {'errorMessage': '정상',
   'countPerPage': '10',
   'totalCount': '1',
   'errorCode': '0',
   'currentPage': '1'},
  'juso': [{'detBdNmList': '',
    'engAddr': '463, Cheongpa-ro, Jung-gu, Seoul',
    'rn': '청파로',
    'emdNm': '중림동',
    'zipNo': '04505',
    'roadAddrPart2': ' (중림동)',
    'emdNo': '02',
    'sggNm': '중구',
    'jibunAddr': '서울특별시 중구 중림동 441 한국경제신문사',
    'siNm': '서울특별시',
    'roadAddrPart1': '서울특별시 중구 청파로 463',
    'bdNm': '한국경제신문사',
    'admCd': '1114017100',
    'udrtYn': '0',
    'lnbrMnnm': '441',
    'roadAddr': '서울특별시 중구 청파로 463 (중림동)',
    'lnbrSlno': '0',
    'buldMnnm': '463',
    'bdKdcd': '0',
    'liNm': '',
    'rnMgtSn': '111403101023',
    'mtYn': '0',
    'bdMgtSn': '1114017100104410000000321',
    'buldSlno': '0'}]}}

In [26]:
# result로부터 bdNm하고 roadAddr 뽑아내기
building_name = result['results']['juso'][0]['bdNm']
road_address = result['results']['juso'][0]['roadAddr']

'서울특별시 중구 청파로 463 (중림동)'

* 5개의 건물명을 리스트로 받아 도로명주소를 구한 뒤 데이터프레임으로 만들어 CSV파일(addr.csv)로 만들기

In [73]:
find = ['한국경제신문사','서울특별시청','종로구청','중구청','효창공원']
blNm = []
roadAddr = []

for each in find:
    url = f'{road_url}?confmKey={road_key}&currentPage=1&countPerPage=10&keyword={quote(each)}&resultType=json'
    result = requests.get(url).json()
    blNm.append(result['results']['juso'][0]['bdNm'])
    roadAddr.append(result['results']['juso'][0]['roadAddr'])

In [74]:
roadAddr = pd.DataFrame({'건물명': blNm,'주소' : roadAddr
})
roadAddr.to_csv('addr.csv',index=False, sep=',',encoding='utf8')